In [1]:
import sys
import os
from pprint import pprint

In [2]:
import tensorflow as tf

In [3]:
import tensorflow.contrib.slim as slim

In [4]:
sys.path.append('../datasets/')
sys.path.append('../nets/')
sys.path.append('../preprocessing/')

In [5]:
import orientset

In [6]:
oriset = orientset.get_split('', '../data/tfrecord/')

In [7]:
import vgg_preprocessing, vgg

In [8]:
# view vgg16 net structure
with tf.Graph().as_default():
    VGG_IMAGE_SIZE = vgg.vgg_16.default_image_size
    image_placeholder = tf.placeholder(tf.float32, [None, VGG_IMAGE_SIZE, VGG_IMAGE_SIZE, 3])
    logits, end_points = vgg.vgg_16(image_placeholder, num_classes=oriset.num_classes, scope='vgg_orient')
end_points

OrderedDict([('vgg_orient/conv1/conv1_1',
              <tf.Tensor 'vgg_orient/conv1/conv1_1/Relu:0' shape=(?, 224, 224, 64) dtype=float32>),
             ('vgg_orient/conv1/conv1_2',
              <tf.Tensor 'vgg_orient/conv1/conv1_2/Relu:0' shape=(?, 224, 224, 64) dtype=float32>),
             ('vgg_orient/pool1',
              <tf.Tensor 'vgg_orient/pool1/MaxPool:0' shape=(?, 112, 112, 64) dtype=float32>),
             ('vgg_orient/conv2/conv2_1',
              <tf.Tensor 'vgg_orient/conv2/conv2_1/Relu:0' shape=(?, 112, 112, 128) dtype=float32>),
             ('vgg_orient/conv2/conv2_2',
              <tf.Tensor 'vgg_orient/conv2/conv2_2/Relu:0' shape=(?, 112, 112, 128) dtype=float32>),
             ('vgg_orient/pool2',
              <tf.Tensor 'vgg_orient/pool2/MaxPool:0' shape=(?, 56, 56, 128) dtype=float32>),
             ('vgg_orient/conv3/conv3_1',
              <tf.Tensor 'vgg_orient/conv3/conv3_1/Relu:0' shape=(?, 56, 56, 256) dtype=float32>),
             ('vgg_orient/conv3/

##  train from scratch

In [9]:
tf.logging.set_verbosity(tf.logging.DEBUG)

In [ ]:
with tf.Graph().as_default():
    # read train data
    data_provider = slim.dataset_data_provider.DatasetDataProvider(oriset)
    image, label = data_provider.get(['image', 'label'])
    VGG_IMAGE_SIZE = vgg.vgg_16.default_image_size
    image = vgg_preprocessing.preprocess_for_train(image, VGG_IMAGE_SIZE, VGG_IMAGE_SIZE)
    
    # batch data
    batch_image, batch_label = tf.train.batch([image, label], batch_size=16, allow_smaller_final_batch=True)
    batch_one_hot_label = slim.one_hot_encoding(batch_label, oriset.num_classes)
    batch_one_hot_label = tf.squeeze(batch_one_hot_label, [1])
    
    print(batch_one_hot_label)
    # create the net
    logits, _ = vgg.vgg_16(batch_image, num_classes=oriset.num_classes, scope='vgg_orient')
    print(logits)
    # create loss
    total_loss = tf.losses.softmax_cross_entropy(batch_one_hot_label, logits)
    
    # create optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    
    # create train_op
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # start to learn
    slim.learning.train(train_op, './logs/', log_every_n_steps=100)

Tensor("Squeeze_1:0", shape=(?, 4), dtype=float32)
Tensor("vgg_orient/fc8/squeezed:0", shape=(?, 4), dtype=float32)
INFO:tensorflow:Restoring parameters from ./logs/model.ckpt-0
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0


In [1]:
# 已经具备保存功能了，但是这里的网络是从头训练
# 事实上，我们不想从头训练，我们想
# 1. 使用已经训练好的部分层的数据
# 2. 对某些层我们不想再训练了

## display trainable variable

In [14]:
with tf.Graph().as_default():
    VGG_IMAGE_SIZE = vgg.vgg_16.default_image_size
    image_placeholder = tf.placeholder(tf.float32, [None, VGG_IMAGE_SIZE, VGG_IMAGE_SIZE, 3])
    
    # create the net
    logits, _ = vgg.vgg_16(image_placeholder, num_classes=oriset.num_classes, scope='vgg_orient')
    # display trainable variable
    trainable_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    print('---- all tainable variables ----')
    print(trainable_variables)
    
    print('---- filtered to train ----')
    # filter the variablee we want
    scopes =['vgg_orient/fc6', 'vgg_orient/fc7', 'vgg_orient/fc8']
    variables_to_train =[]
    for scope in scopes:
        variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
        variables_to_train.extend(variables)
    
    print(variables_to_train)
    # variable_to_train could set by parameters of variables  
    # slim.learning.create_train_op( total_loss, optimizer, variables_to_train = variables_to_train)

---- all tainable variables ----
[<tf.Variable 'vgg_orient/conv1/conv1_1/weights:0' shape=(3, 3, 3, 64) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv1/conv1_1/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv1/conv1_2/weights:0' shape=(3, 3, 64, 64) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv1/conv1_2/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_1/weights:0' shape=(3, 3, 64, 128) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_1/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_2/weights:0' shape=(3, 3, 128, 128) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_2/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv3/conv3_1/weights:0' shape=(3, 3, 128, 256) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv3/conv3_1/biases:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv3/conv3_2/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>, <tf.Va

## load some layers for the first time

In [20]:
with tf.Graph().as_default():
    VGG_IMAGE_SIZE = vgg.vgg_16.default_image_size
    image_placeholder = tf.placeholder(tf.float32, [None, VGG_IMAGE_SIZE, VGG_IMAGE_SIZE, 3])
    
    # create the net
    logits, _ = vgg.vgg_16(image_placeholder, num_classes=oriset.num_classes, scope='vgg_orient')
    
    # scopes for train
    scopes =['vgg_orient/fc6', 'vgg_orient/fc7', 'vgg_orient/fc8']
    
    # get variables to restore
    variables_to_restore = slim.get_variables_to_restore(exclude=scopes)
    print(variables_to_restore)
    for var in variables_to_restore:
        print(var.op.name)
    
    # load the checkpoint which has different name with ours by mapping
    variables_to_restore = { var.op.name.replace('vgg_orient', 'vgg_16') :var for var in variables_to_restore}
    
    checkpoint_path = '../checkpoints/vgg_16.ckpt'
    slim.assign_from_checkpoint_fn(checkpoint_path,variables_to_restore)

[<tf.Variable 'vgg_orient/conv1/conv1_1/weights:0' shape=(3, 3, 3, 64) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv1/conv1_1/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv1/conv1_2/weights:0' shape=(3, 3, 64, 64) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv1/conv1_2/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_1/weights:0' shape=(3, 3, 64, 128) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_1/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_2/weights:0' shape=(3, 3, 128, 128) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv2/conv2_2/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv3/conv3_1/weights:0' shape=(3, 3, 128, 256) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv3/conv3_1/biases:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv3/conv3_2/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>, <tf.Variable 'vgg_orient/conv3/conv3_2/

## combine transfering param and specifing train layers 

In [8]:
with tf.Graph().as_default():
    # read train data
    data_provider = slim.dataset_data_provider.DatasetDataProvider(oriset)
    image, label = data_provider.get(['image', 'label'])
    VGG_IMAGE_SIZE = vgg.vgg_16.default_image_size
    image = vgg_preprocessing.preprocess_for_train(image, VGG_IMAGE_SIZE, VGG_IMAGE_SIZE)
    
    # batch data
    batch_image, batch_label = tf.train.batch([image, label], batch_size=32, allow_smaller_final_batch=True)
    batch_one_hot_label = slim.one_hot_encoding(batch_label, oriset.num_classes)
    batch_one_hot_label = tf.squeeze(batch_one_hot_label, [1])
    
    # create the training net
    logits, _ = vgg.vgg_16(batch_image, num_classes=oriset.num_classes, scope='vgg_16', is_training=True)

    # create loss
    total_loss = tf.losses.softmax_cross_entropy(batch_one_hot_label, logits)
        
    # find the variablee we want to train
    scopes =['vgg_16/fc6', 'vgg_16/fc7', 'vgg_16/fc8']
    variables_to_train =[]
    for scope in scopes:
        variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
        variables_to_train.extend(variables)
    
    # restore the specified layers' parameters
    variables_to_restore = slim.get_variables_to_restore(exclude=scopes)
    variables_to_restore = { var.op.name.replace('vgg_orient', 'vgg_16'):var for var in variables_to_restore}
    
    # create optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.00001, beta1=0.9, beta2=0.999)

    # create train_op
    train_op = slim.learning.create_train_op(total_loss, optimizer, variables_to_train=variables_to_train)
    
    # restore parameters
    init_fn = slim.assign_from_checkpoint_fn('../checkpoints/vgg_16.ckpt',variables_to_restore)
    
    # start to learn
    slim.learning.train(train_op, './logs/', log_every_n_steps=1, init_fn=init_fn)

INFO:tensorflow:Restoring parameters from ./logs/model.ckpt-99
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 100: loss = 0.7037 (66.83 sec/step)
INFO:tensorflow:global step 101: loss = 2.0884 (60.25 sec/step)
INFO:tensorflow:global step 102: loss = 0.7757 (62.47 sec/step)
INFO:tensorflow:global step 103: loss = 1.7277 (62.85 sec/step)
INFO:tensorflow:global step 104: loss = 1.0007 (59.74 sec/step)
INFO:tensorflow:global step 105: loss = 0.8898 (60.81 sec/step)
INFO:tensorflow:global step 106: loss = 1.0514 (56.82 sec/step)
INFO:tensorflow:global step 107: loss = 0.8572 (58.71 sec/step)
INFO:tensorflow:global step 108: loss = 0.9160 (55.96 sec/step)
INFO:tensorflow:global_step/sec: 0.0149993
INFO:tensorflow:global step 109: loss = 0.9540 (56.52 sec/step)
INFO:tensorflow:global step 110: loss = 0.9955 (59.79 sec/step)
INFO:tensorflow:global step 111: loss = 1.2882 (63.94 sec/step)
INFO:tensorflow:global s

KeyboardInterrupt: 